In [17]:
import sys
sys.path.append('./FL_Mnist')
from fl_mnist_implementation_tutorial_utils import *

In [18]:
# %cd ./FL_Mnist

In [19]:
# import zipfile
# zipfile.ZipFile('Mnist.zip').extractall()

# 패키지 가져오기

In [4]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

from fl_mnist_implementation_tutorial_utils import *

# MNIST 데이터 세트 읽기 및 전처리

In [5]:
# def load(paths, verbose=-1):
#     '''expects images for each class in seperate dir, 
#     e.g all digits in 0 class in the directory named 0 '''
#     data = list()
#     labels = list()
#     # loop over the input images
#     for (i, imgpath) in enumerate(paths):
#         # load the image and extract the class labels
#         im_gray = cv2.imread(imgpath, cv2.IMREAD_GRAYSCALE)
#         image = np.array(im_gray).flatten()
#         label = imgpath.split(os.path.sep)[-2]
#         # scale the image to [0, 1] and add to list
#         data.append(image/255)
#         labels.append(label)
#         # show an update every `verbose` images
#         if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
#             print("[INFO] processed {}/{}".format(i + 1, len(paths)))
#     # return a tuple of the data and labels
#     return data, labels

# train test 분할 만들기

In [6]:
# #declear path to your mnist data folder
# img_path = '/app/FL_Mnist/Mnist'

# #get the path list using the path object
# image_paths = list(paths.list_images(img_path))

# #apply our function
# image_list, label_list = load(image_paths, verbose=10000)

# #binarize the labels
# lb = LabelBinarizer()
# label_list = lb.fit_transform(label_list)

# #split data into training and test set
# X_train, X_test, y_train, y_test = train_test_split(image_list, 
#                                                     label_list, 
#                                                     test_size=0.1, 
#                                                     random_state=42)

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train, X_test = X_train/255, X_test/255

# 클라이언트 -> 데이터 샤드

In [7]:
def create_clients(image_list, label_list, num_clients=10, initial='clients'):
    ''' return: a dictionary with keys clients' names and value as 
                data shards - tuple of images and label lists.
        args: 
            image_list: a list of numpy arrays of training images
            label_list:a list of binarized labels for each image
            num_client: number of fedrated members (clients)
            initials: the clients'name prefix, e.g, clients_1 
            
    '''

    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    #randomize the data
    data = list(zip(image_list, label_list))
    random.shuffle(data)

    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))} 

In [8]:
#create clients
clients = create_clients(X_train, y_train, num_clients=10, initial='client')

# 클라이언트 및 테스트 데이터 처리 및 일괄 처리

In [9]:
def batch_data(data_shard, bs=32):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)

In [10]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

50000
10000
50000
10000


In [11]:
#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)

#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

2022-04-15 01:56:25.758070: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-15 01:56:25.758481: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-15 01:56:25.758870: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-15 01:56:25.759296: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/user/miniconda/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/nvidia/lib:/usr/loca

#  (Multi Layer Perceptron) 모델 생성

In [12]:
class SimpleMLP:
    @staticmethod
    def build():
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(32,32,3)))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), kernel_initializer='he_uniform', activation='relu'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(tf.keras.layers.Dropout(0.25))
        model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), kernel_initializer='he_uniform', activation='relu'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), kernel_initializer='he_uniform', activation='relu'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(tf.keras.layers.Dropout(0.25))
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(512, activation='relu'))
        model.add(tf.keras.layers.Dropout(0.5))
        model.add(tf.keras.layers.Dense(100, activation='softmax'))
        return model

In [13]:
lr = 0.01 
comms_round = 100
loss='sparse_categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(lr=lr, 
                decay=lr / comms_round, 
                momentum=0.9
               )

/home/user/miniconda/lib/python3.9/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


# 모델 집계 (연동 평균)

In [14]:
# 클라이언트 데이터의 스케일링 계수 
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
#     print("local_count: ", local_count)
#     print("global_count: ", global_count)
    return local_count/global_count

# 위에서 구한 스케일링 계수를 이용해 스케일된 weight 구하기
def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final


# 
def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

# 연합 모델 교육

In [15]:
#initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build()
opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, decay=1e-6, nesterov=True)
global_model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
cnt = 0
#commence global training loop
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    
    #loop through each client and create new local model
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build()
        local_model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
#     for(X_test, Y_test) in test_batched:
#         global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)
    score = global_model.evaluate(X_test, y_test, verbose=0)
    print(f'comm_round: {cnt} | global_acc: {round(score[1], 4)}% | global_loss: {round(score[0], 4)}')
    cnt += 1

comm_round: 0 | global_acc: 0.1174% | global_loss: 3.1284
comm_round: 1 | global_acc: 0.3024% | global_loss: 1.9795
comm_round: 2 | global_acc: 0.463% | global_loss: 1.5322
comm_round: 3 | global_acc: 0.5363% | global_loss: 1.3218
comm_round: 4 | global_acc: 0.4842% | global_loss: 1.4937
comm_round: 5 | global_acc: 0.4347% | global_loss: 1.7468
comm_round: 6 | global_acc: 0.5157% | global_loss: 1.474
comm_round: 7 | global_acc: 0.5647% | global_loss: 1.2387
comm_round: 8 | global_acc: 0.6104% | global_loss: 1.1093
comm_round: 9 | global_acc: 0.6521% | global_loss: 0.9928
comm_round: 10 | global_acc: 0.6365% | global_loss: 1.0423
comm_round: 11 | global_acc: 0.6753% | global_loss: 0.9265
comm_round: 12 | global_acc: 0.6779% | global_loss: 0.9206
comm_round: 13 | global_acc: 0.7024% | global_loss: 0.8402
comm_round: 14 | global_acc: 0.7079% | global_loss: 0.8323
comm_round: 15 | global_acc: 0.7131% | global_loss: 0.8106
comm_round: 16 | global_acc: 0.7258% | global_loss: 0.7805
comm_roun